### Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

%matplotlib inline
sys.path.append("..")

from utils import label_map_util
from utils import visualization_utils as vis_util
from utils import path_detection_utils as path_util
from utils import data_collection_util as data_util

### Loading in model 

In [2]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

### Object Detection on specific frame

In [3]:
path1 = 'video_frames/frame199.jpg'
path2 = 'video_frames/frame200.jpg'
path3 = 'video_frames/frame201.jpg'

path4 = 'test_images/pic.jpg'

image1 = Image.open(path1)
width1, height1 = image1.size
image_np1 = path_util.load_image_into_numpy_array(image1)
boxes1, scores1, classes1 = path_util.get_segmentation(path1,detection_graph, category_index, False)
human_boxes1, human_scores1 = path_util.remove_non_human(boxes1, scores1, classes1)
good_boxes1, good_scores1 = path_util.remove_low_prob(human_boxes1, human_scores1)
good_boxes1, good_scores1 = path_util.remove_poorly_sized_people(good_boxes1, good_scores1)

image2 = Image.open(path2)
width2, height2 = image2.size
image_np2 = path_util.load_image_into_numpy_array(image2)
boxes2, scores2, classes2 = path_util.get_segmentation(path2,detection_graph, category_index, False)
human_boxes2, human_scores2 = path_util.remove_non_human(boxes2, scores2, classes2)
good_boxes2, good_scores2 = path_util.remove_low_prob(human_boxes2, human_scores2)
good_boxes2, good_scores2 = path_util.remove_poorly_sized_people(good_boxes2, good_scores2)

image3 = Image.open(path3)
width3, height3 = image3.size
image_np3 = path_util.load_image_into_numpy_array(image3)
boxes3, scores3, classes3 = path_util.get_segmentation(path3,detection_graph, category_index, False)
human_boxes3, human_scores3 = path_util.remove_non_human(boxes3, scores3, classes3)
good_boxes3, good_scores3 = path_util.remove_low_prob(human_boxes3, human_scores3)
good_boxes3, good_scores3 = path_util.remove_poorly_sized_people(good_boxes3, good_scores3)

In [3]:
stuff = data_util.collect_data('video_frames/frame', 0, 5000,5002, detection_graph, category_index)

utils/data_collection_util.py:162: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if flow == "nah":


In [16]:
import json

In [19]:
with open('data5000_labels.json', 'w') as fp:
    json.dump(stuff[1], fp)

In [35]:
stuff2 = []
for i in stuff[0]:
    entry = i
    #entry['box'] = entry['box'].tolist()
    #entry['flow'] = entry['flow'].tolist()
    try:
        entry['flow'] = [float(i) for i in entry['flow']]
    stuff2.append(entry)

SyntaxError: invalid syntax (<ipython-input-35-b384cc302274>, line 8)

In [33]:
type(stuff2[0]['flow'][0])

numpy.float32

In [23]:
with open('data5000_everything.json', 'w') as fp:
    json.dump(stuff2, fp)

TypeError: -3.2773499 is not JSON serializable

In [13]:
person_to_boxes = defaultdict(list)
person_to_flow = defaultdict(list)
person_to_flow_x = defaultdict(list)
for entry in stuff[0]:
    if stuff[1][entry['person_id']] != "unknown":
        person_to_boxes[entry['person_id']].append(entry['box'])
        person_to_flow[entry['person_id']].append(entry['direction'])
        person_to_flow_x[entry['person_id']].append(entry['flow'][0])

In [14]:
correct = 0
total = 0
incorrect1 = []
for i in person_to_boxes:
    label = stuff[1][i][0]
    box = person_to_boxes[i][0]
    center_x = (box[3]+box[1])/2.0
    if center_x < 0.5 and label == "l":
        correct+=1
    elif center_x >=0.5 and label == "r":
        correct+=1
    else:
        incorrect1.append(i)
    total+=1
print(correct/ (1.0*total))
print(total)
print(incorrect1)

0.740634005764
347
[2061, 2063, 2064, 31, 2137, 2155, 2156, 2295, 254, 256, 2315, 274, 286, 296, 299, 334, 386, 409, 415, 451, 506, 515, 517, 521, 538, 546, 555, 595, 610, 613, 636, 819, 823, 842, 844, 852, 854, 902, 928, 964, 965, 976, 982, 1000, 1048, 1055, 1062, 1123, 1155, 1247, 1251, 1311, 1313, 1316, 1320, 1321, 1357, 1394, 1435, 1481, 1489, 1537, 1554, 1595, 1601, 1612, 1621, 1633, 1639, 1641, 1650, 1654, 1656, 1664, 1696, 1698, 1700, 1701, 1766, 1772, 1778, 1812, 1820, 1855, 1878, 1881, 1884, 1995, 2023, 2037]


In [15]:
correct = 0
total = 0
incorrect2 = []
for i in person_to_flow:
    label = stuff[1][i][0]
    if person_to_flow_x[i][0] <= 0 and label == "l":
        correct+=1
    elif person_to_flow_x[i][0] > 0 and label == "r":
        correct+=1
    else:
        incorrect2.append(i)
    total+=1
print(correct/ (1.0*total))
print(incorrect2)

0.423631123919
[2061, 2063, 2065, 2069, 22, 24, 2090, 2099, 2120, 2137, 2139, 2153, 2155, 2156, 2157, 2163, 130, 138, 2189, 160, 164, 173, 177, 2236, 2246, 2281, 2291, 2295, 254, 256, 263, 2315, 2318, 2321, 274, 276, 279, 282, 286, 296, 299, 322, 326, 369, 395, 409, 453, 467, 492, 506, 515, 516, 517, 518, 521, 525, 527, 533, 538, 548, 553, 556, 597, 598, 637, 656, 693, 740, 760, 785, 815, 819, 823, 824, 828, 838, 842, 850, 852, 854, 879, 883, 887, 891, 898, 902, 911, 928, 939, 948, 951, 965, 968, 969, 970, 971, 972, 974, 976, 978, 979, 995, 998, 1004, 1005, 1018, 1034, 1037, 1052, 1055, 1155, 1158, 1166, 1180, 1182, 1183, 1185, 1186, 1189, 1194, 1199, 1215, 1223, 1225, 1228, 1232, 1242, 1245, 1247, 1248, 1249, 1250, 1251, 1252, 1256, 1262, 1267, 1275, 1278, 1283, 1284, 1287, 1291, 1292, 1293, 1311, 1313, 1316, 1355, 1361, 1384, 1392, 1394, 1402, 1426, 1435, 1442, 1446, 1448, 1458, 1460, 1473, 1474, 1475, 1476, 1481, 1483, 1489, 1490, 1537, 1554, 1595, 1612, 1622, 1641, 1642, 1650, 1654

In [ ]:
person_to_flow[]

In [21]:
s1 = set(incorrect1)
s2 = set(incorrect2)
s1&s2

{254, 256, 274, 286, 296, 299}

In [30]:
path1 = 'video_frames/frame454.jpg'
path2 = 'video_frames/frame459.jpg'
path3 = 'video_frames/frame201.jpg'

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


In [31]:
cv2.__version__

'3.3.0'

In [42]:
def get_optical_flow(path1, path2):
    print(path1)
    print(path2)
    frame1 = cv2.imread(path1)
    frame2 = cv2.imread(path2)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255
    prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    nxt = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    #p0 = cv2.goodFeaturesToTrack(prvs, mask = None, **feature_params)
    #p1, st, err = cv2.calcOpticalFlowPyrLK(prvs,nxt, p0, None, **lk_params)
    #mask = np.zeros_like(frame1)
    #print(np.shape(prvs))
    #print(np.shape(nxt))
    #flow = cv2.calcOpticalFlowFarneback(prvs,nxt,None, 0.5,3.0,15.0,3.0,5.0,1.2,0.0)
    #flow = cv2.calcOpticalFlowFarneback(prvs,nxt,0.5,1,3,15,3,5,1,-1)
    flow = cv2.calcOpticalFlowFarneback(prvs,nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

In [43]:
get_optical_flow(path1, path2)

video_frames/frame454.jpg
video_frames/frame459.jpg


array([[[  2.54660342e-02,  -3.97697417e-03],
        [  3.42693999e-02,  -6.11932017e-03],
        [  4.30045761e-02,  -8.95587169e-03],
        ..., 
        [  2.52231769e-03,  -4.13058966e-04],
        [  1.15387829e-03,  -2.12281870e-04],
        [  3.83248174e-04,  -1.19668846e-04]],

       [[  4.05092277e-02,  -2.87246914e-03],
        [  5.79183660e-02,  -4.86385077e-03],
        [  7.50252232e-02,  -7.79721839e-03],
        ..., 
        [  3.08876787e-03,  -3.15024838e-04],
        [  1.55377446e-03,  -9.93537251e-05],
        [  6.31574367e-04,  -3.99987548e-05]],

       [[  6.74146861e-02,  -3.83336865e-03],
        [  9.59740505e-02,  -6.82021398e-03],
        [  1.21963412e-01,  -1.10972868e-02],
        ..., 
        [  4.21585655e-03,  -3.97547701e-04],
        [  2.32851016e-03,  -9.52896808e-05],
        [  1.12048618e-03,  -1.74001689e-05]],

       ..., 
       [[  9.86305968e-05,   6.97549649e-06],
        [  1.63860997e-04,   4.50908556e-05],
        [  2.578088

In [ ]:
person_to_boxes = defaultdict(list)
for entry in stuff[0]:
    if stuff[1][entry['person_id']] != "unknown":
        person_to_boxes[entry['person_id']].append(entry['box'])

In [ ]:
correct = 0
for i in person_to_boxes:
    label = stuff[1][i][0]
    box = person_to_boxes[i][0]
    center_x = (box[3]+box[1])/2.0
    if center_x < 0.5 and label == "l":
        correct+=1
    if center_x >=0.5 and label == "r":
        correct+=1
print(correct)